In [1]:
#| default_exp updates
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Atualização

> Este módulo atualiza as bases. Executa as queries sql do STEL, RADCOM e baixa os arquivos de estações e plano básico do MOSAICO.

In [2]:
#| export
import os
from decimal import Decimal, getcontext
from typing import Union
import gc

import pandas as pd
import pyodbc
from rich.console import Console
from rich import print
from pyarrow import ArrowInvalid, ArrowTypeError
from fastcore.xtras import Path
from fastcore.test import test_eq
from fastcore.parallel import parallel
import pyodbc
from pymongo import MongoClient
from dotenv import load_dotenv

from extracao.icao import get_icao
from extracao.aisgeo import get_aisg
from extracao.aisweb import get_aisw
from extracao.redemet import get_redemet
from extracao.constants import *
from extracao.format import parse_bw, merge_close_rows, _read_df


getcontext().prec = 5
load_dotenv()

True

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [3]:
#| export
def connect_db(
    server: str = "ANATELBDRO05",  # Servidor do Banco de Dados
    database: str = "SITARWEB",  # Nome do Banco de Dados
    trusted_conn: str = "yes",  # Conexão Segura: yes | no
    mult_results: bool = True,  # Múltiplos Resultados
) -> pyodbc.Connection:
    """Conecta ao Banco `server` e retorna o 'cursor' (iterador) do Banco"""
    return pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        f"Server={server};"
        f"Database={database};"
        f"Trusted_Connection={trusted_conn};"
        f"MultipleActiveResultSets={mult_results};",
        timeout=TIMEOUT,
    )


In [4]:
#echo: false
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (SQL_RADCOM,SQL_STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [5]:
#| eval: false
test_connection()

In [6]:
#| export
def clean_mosaico(
    df: pd.DataFrame,  # DataFrame com os dados de Estações e Plano_Básico mesclados
    pasta: Union[
        str, Path
    ],  # Pasta com os dados de municípios para imputar coordenadas ausentes
) -> pd.DataFrame:  # DataFrame com os dados mesclados e limpos
    """Clean the merged dataframe with the data from the MOSAICO page"""
    df = df[
        df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)
    ].reset_index(drop=True)
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    df.loc["Frequência"] = df.Frequência.astype("str").str.replace(",", ".")
    df = df[df.Frequência.notna()].reset_index(drop=True)
    df.loc["Frequência"] = df.Frequência.astype("float")
    df.loc[df.Num_Serviço == "205", "Frequência"] = df.loc[
        df.Num_Serviço == "205", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    df.loc[:, "Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    return df


## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [7]:
#| export
def _save_df(df: pd.DataFrame, folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Format, Save and return a dataframe"""
    df = df.copy()  # Impedir a alteração do df original
    for c in df.columns:
        df[c] = df[c].astype("string").str.lstrip().str.rstrip()
    df = df.drop_duplicates(keep="first").reset_index(drop=True)
    if "Código_Município" in df:
        df = df[df.Código_Município.notna()].reset_index(drop=True)
    try:
        file = Path(f"{folder}/{stem}.parquet.gzip")
        df.to_parquet(file, compression="gzip", index=False)
    except (ArrowInvalid, ArrowTypeError) as e:
        raise e(f"Não possível salvar o arquivo parquet {file}")
    return df


In [8]:
# | export
def update_radcom(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `RADCOM`, com tratamento de erro de conectividade."""
    console = Console()
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="monkey"
    ) as status:
        try:
            return _extract_radcom(conn, folder)
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e


def _extract_radcom(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    df = pd.read_sql_query(SQL_RADCOM, conn)
    df["Entidade"] = df.Entidade.str.rstrip().str.lstrip()
    df["Num_Serviço"] = "231"
    df["Classe_Emissão"] = pd.NA
    df["Largura_Emissão(kHz)"] = "256"
    df["Validade_RF"] = pd.NA
    df["Status"] = "RADCOM"
    df["Fonte"] = "SRD"
    df["Multiplicidade"] = "1"
    a = df.Situação.isna()
    df.loc[a, "Classe"] = df.loc[a, "Fase"]
    df.loc[~a, "Classe"] = (
        df.loc[~a, "Fase"].astype("string")
        + "-"
        + df.loc[~a, "Situação"].astype("string")
    )
    df.drop(["Fase", "Situação"], axis=1, inplace=True)
    df = df.loc[:, COLUNAS]
    return _save_df(df, folder, "radcom")


In [9]:
#| eval: false
import warnings
import os
# warnings.filterwarnings("ignore", message='install "ipywidgets" for Jupyter support')
warnings.filterwarnings("ignore")

In [10]:
%%time
#| eval: false
folder = Path.cwd().parent / 'dados'
conn = connect_db()
radcom = update_radcom(conn, folder)
radcom

Output()

CPU times: total: 797 ms
Wall time: 3.02 s


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,1008210959,Turilândia,2112456,MA,-2.228611111111,-45.306666666666665,P,231,<NA>,256,<NA>,RADCOM,SRD,1
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,699491851,Alvorada de Minas,3102407,MG,-18.734166666666667,-43.36472222222217,3,231,<NA>,256,<NA>,RADCOM,SRD,1
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,699359830,Congonhas do Norte,3118106,MG,-18.812777777777665,-43.673611111111,3,231,<NA>,256,<NA>,RADCOM,SRD,1
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,690859562,Dom Joaquim,3122603,MG,-18.95,-43.266666666666666,3,231,<NA>,256,<NA>,RADCOM,SRD,1
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,1008401606,Castanheira,5102850,MT,-11.137222222222167,-58.61333333333334,P-A,231,<NA>,256,<NA>,RADCOM,SRD,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4934,107.9,ASSOCIAÇÃO CULT COMUNIT MORUMBI,50407431578,697707695,São José dos Campos,3549904,SP,-23.267777777777667,-45.8975,3-B,231,<NA>,256,<NA>,RADCOM,SRD,1
4935,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50413097013,692270272,São José dos Campos,3549904,SP,-23.5530555555555,-45.8705555555555,3,231,<NA>,256,<NA>,RADCOM,SRD,1
4936,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50434484237,692270272,São José dos Campos,3549904,SP,-23.5530555555555,-45.8705555555555,3,231,<NA>,256,<NA>,RADCOM,SRD,1
4937,107.9,ASSOCIACAO COMUNITARIA CULTURAL DE MUSICA E CI...,50406778205,693049723,São José dos Campos,3549904,SP,-23.191944444444335,-45.87527777777767,3,231,<NA>,256,<NA>,RADCOM,SRD,1


In [11]:
#|export
def update_stel(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`, com tratamento de erro de conectividade."""
    console = Console()
    with console.status(
        "[red]Lendo o Banco de Dados do STEL",
        spinner="grenade",
    ) as status:
        try:
            return _extract_stel(conn, folder)
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e


def _extract_stel(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`"""
    stel = pd.read_sql_query(SQL_STEL, conn)
    stel["Status"] = "L"
    stel["Entidade"] = stel.Entidade.str.rstrip().str.lstrip()
    stel["Fonte"] = "STEL"
    stel.loc[:, ["Largura_Emissão(kHz)", "_"]] = (
        stel.Largura_Emissão.fillna("").apply(parse_bw).tolist()
    )
    stel.drop(["Largura_Emissão", "_"], axis=1, inplace=True)
    stel.loc[:, "Validade_RF"] = stel.Validade_RF.astype("string").str.slice(0, 10)
    stel.loc[stel.Unidade == "kHz", "Frequência"] = stel.loc[
        stel.Unidade == "kHz", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    stel.loc[stel.Unidade == "GHz", "Frequência"] = stel.loc[
        stel.Unidade == "GHz", "Frequência"
    ].apply(lambda x: Decimal(x) * Decimal(1000))
    stel.drop("Unidade", axis=1, inplace=True)
    stel["Multiplicidade"] = 1
    stel = stel.loc[:, COLUNAS]
    return _save_df(stel, folder, "stel")


In [12]:
%%time
#| eval: false
stel = update_stel(conn, folder)
stel

Output()

CPU times: total: 6.11 s
Wall time: 9.66 s


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,10512,REDE 21 COMUNICACOES S.A.,50000257176,1129953,São Paulo,3550308,SP,-23.554722222222168,-46.664444444444335,TX,251,F3F,18000.0,2018-10-05,L,STEL,1
1,10620,REDE 21 COMUNICACOES S.A.,50000257176,1129953,São Paulo,3550308,SP,-23.554722222222168,-46.664444444444335,TX,251,F3F,18000.0,2018-10-05,L,STEL,1
2,10755,Cirion Technologies do Brasil Ltda,50004957806,535827539,São Paulo,3550308,SP,-23.566388888888834,-46.648611111111,<NA>,046,<NA>,-1,2035-02-16,L,STEL,1
3,10835,UNIVERSAL TELECOM S.A.,50014055481,1000279658,Petrópolis,3303906,RJ,-22.5391055555555,-43.14901666666667,FX,046,D7D,40000.0,2029-02-17,L,STEL,1
4,10835,NETWAVE TELECOMUNICACOES LTDA.,50410866563,699930642,Valinhos,3556206,SP,-22.947027777777667,-46.903863888888836,FX,046,D7W,28000.0,2028-10-01,L,STEL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67282,25.375,REDE D OR SÃO LUIZ S.A.,50411840118,1000703280,São Paulo,3550308,SP,-23.5905,-46.67327777777766,TX,060,F9W,25.0,2034-11-03,L,STEL,1
67283,25.375,SOC BENEFICIENTE DE SENHORAS HOSPITAL SIRIO LI...,50401547558,696670844,São Paulo,3550308,SP,-23.557694444444333,-46.654027777777664,TX,060,F9W,25.0,2034-09-28,L,STEL,1
67284,25.375,SOC BENEFICIENTE DE SENHORAS HOSPITAL SIRIO LI...,50401547558,1000851637,São Paulo,3550308,SP,-23.557416666666665,-46.653111111111,TX,060,F9W,25.0,2034-09-28,L,STEL,1
67285,25.375,SOCIEDADE BENEFICENTE SAO CAMILO,50410364975,1010448703,São Paulo,3550308,SP,-23.533766666666665,-46.68906666666667,TX,060,F9W,25.0,2023-05-09,L,STEL,1


In [13]:
#|export
def update_mosaico(
    mongo_client: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Efetua a query na tabela de Radiodifusão no banco mongoDB `mongo_client` e atualiza o arquivo local"""
    console = Console()
    with console.status(
        "Consolidando os dados do Mosaico...", spinner="runner"
    ) as status:

        database = mongo_client["sms"]
        collection = database["srd"]
        list_data = list(collection.find(MONGO_SRD, projection=COLS_SRD.keys()))
        mosaico = pd.json_normalize(list_data)
        mosaico = mosaico.drop(columns=["estacao"])
        mosaico = mosaico[list(COLS_SRD.keys())]
        mosaico.rename(COLS_SRD, axis=1, inplace=True)
        mosaico = clean_mosaico(mosaico, folder)
        mosaico["Fonte"] = "MOS"
        mosaico["Num_Serviço"].fillna("", inplace=True)
        mosaico.loc[:, ["Largura_Emissão(kHz)", "Classe_Emissão"]] = (
            mosaico.Num_Serviço.astype("string")
            .fillna("")
            .map(BW_MAP)
            .apply(parse_bw)
            .tolist()
        )
        mosaico.loc[mosaico.Classe_Emissão == "", "Classe_Emissão"] = pd.NA
        mosaico["Multiplicidade"] = 1
        mosaico = mosaico.loc[:, COLUNAS]
    return _save_df(mosaico, folder, "mosaico")


In [14]:
#|eval: false
uri = os.environ['MONGO_URI']
mongo_client = MongoClient(uri)
mongo_client.server_info()

{'version': '4.0.5',
 'gitVersion': '3739429dd92b92d1b0ab120911a23d50bf03c412',
 'targetMinOS': 'Windows 7/Windows Server 2008 R2',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 0, 5, 0],
 'openssl': {'running': 'Windows SChannel'},
 'buildEnvironment': {'distmod': '2008plus-ssl',
  'distarch': 'x86_64',
  'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'ccflags': '/nologo /EHsc /W3 /wd4355 /wd4800 /wd4267 /wd4244 /wd4290 /wd4068 /wd4351 /wd4373 /we4013 /we4099 /we4930 /WX /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /Zc:rvalueCast /Zc:strictStrings /volatile:iso /Gw /Gy /Zc:inline',
  'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'cxxflags': '/TP',
  'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF',
  'target_arch': 'x86_64',
  'target_os': 'windows'},
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 '

In [15]:
%%time
#|eval: false
mosaico = update_mosaico(mongo_client, folder)
display(mosaico)

Output()

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,207.0,REDE DE COMUNICACOES ACREANA LTDA,50442889933,<NA>,Cruzeiro do Sul,1200203,AC,<NA>,<NA>,A,248,<NA>,6000.0,<NA>,TV-C1,MOS,1
1,539.0,X-MEDIAGROUP S.A.,50410887137,<NA>,Mâncio Lima,1200336,AC,<NA>,<NA>,C,248,<NA>,6000.0,<NA>,TV-C1,MOS,1
2,79.0,TELEVISAO OESTE BAIANO LTDA,06030116240,322647029,Barreiras,2903201,BA,-12.1013888888888333,-44.9936111111110000,A,248,<NA>,6000.0,2023-12-31,TV-C4,MOS,1
3,69.0,TELEVISAO SANTA CRUZ LTDA,06020355110,322623553,Itabuna,2914802,BA,-14.7794444444443333,-39.2622222222221666,A,248,<NA>,6000.0,2023-12-31,TV-C4,MOS,1
4,177.0,TV CABRALIA LTDA,06020354903,322623537,Itabuna,2914802,BA,-14.7833333333333333,-39.2833333333333333,B,248,<NA>,6000.0,2023-12-31,TV-C4,MOS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30358,635.0,REDE RONDONIA DE COMUNICACAO LTDA,50444067248,<NA>,Codajás,1301308,AM,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
30359,629.0,REDE RONDONIA DE COMUNICACAO LTDA,50444067400,<NA>,Parintins,1303403,AM,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
30360,91.1,SISTEMA NOROESTE DE COMUNICACAO LTDA EPP,50407817506,1007790129,Piacatu,3537701,SP,<NA>,<NA>,C,230,<NA>,256.0,2029-01-08,FM-C4,MOS,1
30361,557.0,KAKE TV LTDA,50444198776,<NA>,Espigão D'Oeste,1100098,RO,<NA>,<NA>,C,801,<NA>,5700.0,2043-01-12,TV-C2,MOS,1


CPU times: total: 3.44 s
Wall time: 4.17 s


In [16]:
#| export 
def update_aero(
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a base de dados de emissões da aeronáutica"""
    icao = get_icao()
    aisw = get_aisw()
    aisg = get_aisg()
    redemet = get_redemet()
    radares = pd.read_excel(os.environ["PATH_RADAR"])
    for df in [aisw, aisg, redemet, radares]:
        icao = merge_close_rows(icao, df)
    return _save_df(icao, folder, "aero")


In [17]:
# %%time
# aero = update_aero(folder)
# display(aero)

In [18]:
#| export
def update_telecom(
    mongo_client: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Efetua a query na tabela `licenciamento` no banco mongoDB `mongo_client` e atualiza o arquivo local"""

    database = mongo_client["sms"]
    collection = database["licenciamento"]
    query = collection.find(
        MONGO_TELECOM, projection={k: 1.0 for k in COLS_TELECOM.keys()}
    )
    print(
        "[red] :warning: Executando a query na base licenciamento do Mosaico, processo demorado! :warning:"
    )
    df = pd.DataFrame([c for c in query], columns=COLS_TELECOM.keys(), dtype="string")
    path_cache = Path(f"{folder}/telecom_raw.parquet.gzip")
    path_out = Path(f"{folder}/telecom.parquet.gzip")
    if path_cache.is_file():
        cache_df = pd.read_parquet(path_cache)
        if df.equals(cache_df) and path_out.is_file():
            del df
            gc.collect()
            return pd.read_parquet(path_out)
    df.to_parquet(path_cache, compression="gzip", index=False)
    return _process_telecom(df, folder)


def _process_telecom(
    df: pd.DataFrame,  # Dataframe não processado de dados do Mosaico
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:
    """Formata e pós-processa e mescla os dados de Telecomunicações do Mosaico"""
    # df.drop("_id", axis=1, inplace=True)
    df.rename(COLS_TELECOM, axis=1, inplace=True)
    df["Designacao_Emissão"] = df.Designacao_Emissão.str.replace(",", " ")
    df["Designacao_Emissão"] = (
        df.Designacao_Emissão.str.strip().str.lstrip().str.rstrip().str.upper()
    )
    df["Designacao_Emissão"] = df.Designacao_Emissão.str.split(" ")
    df = df.explode("Designacao_Emissão")
    df.loc[df.Designacao_Emissão == "/", "Designacao_Emissão"] = ""
    df.loc[:, ["Largura_Emissão(kHz)", "Classe_Emissão"]] = df.Designacao_Emissão.apply(
        parse_bw
    ).tolist()
    df.drop("Designacao_Emissão", axis=1, inplace=True)
    subset = [
        "Entidade",
        "Longitude",
        "Latitude",
        "Classe",
        "Frequência",
        "Num_Serviço",
        "Largura_Emissão(kHz)",
        "Classe_Emissão",
    ]
    df.dropna(subset=subset, axis=0, inplace=True)
    df_sub = (
        df[~df.duplicated(subset=subset, keep="first")].reset_index(drop=True).copy()
    )
    df_sub["Multiplicidade"] = (
        df.groupby(subset, sort=False).count()["Número_Estação"]
    ).tolist()
    df_sub["Status"] = "L"
    df_sub["Fonte"] = "MOS"
    del df
    gc.collect()
    df_sub = df_sub.reset_index()
    df_sub = df_sub.loc[:, COLUNAS]
    return _save_df(df_sub, folder, "telecom")


In [19]:
# %%time
# #| eval: false
# telecom = update_telecom(mongo_client, folder)
# telecom
telecom = pd.read_parquet(folder / 'telecom.parquet.gzip')

In [72]:
#| export
def validar_coords(
    row: pd.Series,  # Linha de um DataFrame
    connector: pyodbc.Connection = None,  # Conector de Banco de Dados
) -> tuple:  # DataFrame com dados do município
    """Valida os dados de coordenadas e município em `row` no polígono dos municípios em banco corporativ do IBGE"""

    mun, cod, lat, long = (
        row.Município,
        row.Código_Município,
        row.Latitude,
        row.Longitude,
    )
    is_valid = '-1'
    conn = connect_db() if connector is None else connector
    crsr = conn.cursor()
    sql = SQL_VALIDA_COORD.format(long, lat, cod)
    crsr.execute(sql)
    result = crsr.fetchone()
    if result is not None:
        mun = result.NO_MUNICIPIO
        lat = result.NU_LATITUDE
        long = result.NU_LONGITUDE
        is_valid = result.COORD_VALIDA
    if connector is None:
        del conn
    return [str(mun), str(lat), str(long), str(is_valid)]


In [21]:
# | export
def _validar_coords_base(
    df: pd.DataFrame,  # DataFrame com os dados da Anatel
    cache_df: pd.DataFrame,  # DataFrame validado anteriormente, usado como cache
) -> pd.DataFrame:  # DataFrame com as coordenadas validadas na base do IBGE
    """Valida as coordenadas consultado a Base Corporativa do IBGE, excluindo o que já está no cache na versão anterior"""
    pass


In [80]:
# | export
def update_base(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    clientMongoDB: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    # sourcery skip: use-fstring-for-concatenation
    """Wrapper que atualiza opcionalmente lê e atualiza as 4 bases indicadas anteriormente, as combina e salva o arquivo consolidado na folder `folder`"""
    stel = update_stel(conn, folder)
    radcom = update_radcom(conn, folder)
    mosaico = update_mosaico(clientMongoDB, folder)
    telecom = update_telecom(clientMongoDB, folder)

    base = (
        pd.concat([mosaico, radcom, stel, telecom])
        .sort_values(["Frequência", "Latitude", "Longitude"])
        .reset_index(drop=True)
    )

    for c in base.columns:
        base[c] = base[c].astype("string")

    base.loc[:, ["Latitude", "Longitude"]].fillna("0", inplace=True)

    df_cache = _read_df(folder, 'base')

    ibge = ["Município_IBGE", "Latitude_IBGE", "Longitude_IBGE", "Coords_Valida_IBGE"]

    df_cache = pd.concat([df_cache, base]).drop_duplicates(subset=base.columns, keep='first').reset_index(drop=True)

    subset = df_cache.Coords_Valida_IBGE.isna()

    df_cache.loc[:, ["Latitude", "Longitude"]].fillna("-1", inplace=True)

    linhas = list(df_cache.loc[subset, ['Município', 'Código_Município', 'Latitude', 'Longitude']].itertuples())

    df_cache.loc[subset, ibge] = parallel(validar_coords, linhas, threadpool=True, n_workers=20, progress=True) 

    return _save_df(df_cache, folder, "base")